In [1]:
import pandas as pd
import httplib2
from pathlib import Path
from bs4 import BeautifulSoup, SoupStrainer
pd.set_option('display.max_rows', 200)

In [2]:
def get_data():
    """
    The link to the Sponsor list  is updated. This function is designed to fetch the newest version
    """
    # Get webpage
    http = httplib2.Http()
    status, response = http.request('https://www.gov.uk/government/publications/register-of-licensed-sponsors-workers')
    
    # Search for the link
    for link in BeautifulSoup(response, parse_only=SoupStrainer('a')):
        if link.has_attr('href') and link['href'].startswith("https://assets.publishing.service.gov.uk/"):
            file_name = link['href'].split('/')[-1]
            print("\nLast updated:", file_name[:10])
            
            # Fetch the latest data, cache and use old data if there is no update
            if Path(file_name).is_file():
                df = pd.read_csv(file_name)
            else:
                df = pd.read_csv(link['href'])
                df.to_csv(file_name)
                
    return df

In [3]:
# Fetch data
data = get_data()


Last updated: 2023-03-27


In [4]:
def search(word, based_on=None, route=None):
    
    if based_on is None:
        based_on = "org"
    if route is None:
        route = "Skilled Worker"
        
    """
    based_on: Use "org", "toc" or "c" to search based on "Organisation Name", "Town/City" or "County" repectively. 
    word: This is the text to be searched
    route: Any of the elements present in data['Route'].unique(). default is "Skilled Worker".
    
    returns: A dataframe showing the results of the search.
    """
    
    expand_arg = {'org': "Organisation Name", 'toc': "Town/City", 'c': "County"}
    column = expand_arg[based_on]
    df = data.dropna(subset=[column])
    sponsor = df[df[column].str.contains(word, case=False)]
    return sponsor[sponsor["Route"] == route]

In [5]:
response = search(word="University of E", based_on="org", route="Skilled Worker")
response

,Organisation Name,Town/City,County,Type & Rating,Route
68977,University of East Anglia,Norwich,Norfolk,Worker (A rating),Skilled Worker
68979,University of East London,London,NaN,Worker (A rating),Skilled Worker
68981,University of Edinburgh,Edinburgh,NaN,Worker (A (SME+)),Skilled Worker
68985,University of Essex,Colchester,Essex,Worker (A rating),Skilled Worker
68989,University of Exeter,Exeter,Devon,Worker (A (SME+)),Skilled Worker


In [6]:
var = response.to_dict('records')

In [7]:
import json
from pprint import pprint

json.dumps({str(i) : response.to_dict('records')[i] for i in range(len(response))})


'{"0": {"Organisation Name": "University of East Anglia", "Town/City": "Norwich", "County": "Norfolk", "Type & Rating": "Worker (A rating)", "Route": "Skilled Worker"}, "1": {"Organisation Name": "University of East London", "Town/City": "London", "County": NaN, "Type & Rating": "Worker (A rating)", "Route": "Skilled Worker"}, "2": {"Organisation Name": "University of Edinburgh", "Town/City": "Edinburgh", "County": NaN, "Type & Rating": "Worker (A (SME+))", "Route": "Skilled Worker"}, "3": {"Organisation Name": "University of Essex", "Town/City": "Colchester", "County": "Essex", "Type & Rating": "Worker (A rating)", "Route": "Skilled Worker"}, "4": {"Organisation Name": "University of Exeter", "Town/City": "Exeter", "County": "Devon", "Type & Rating": "Worker (A (SME+))", "Route": "Skilled Worker"}}'

In [8]:
response.to_json()

'{"Organisation Name":{"68977":"University of East Anglia","68979":"University of East London","68981":"University of Edinburgh","68985":"University of Essex","68989":"University of Exeter"},"Town\\/City":{"68977":"Norwich","68979":"London","68981":"Edinburgh","68985":"Colchester","68989":"Exeter"},"County":{"68977":"Norfolk","68979":null,"68981":null,"68985":"Essex","68989":"Devon"},"Type & Rating":{"68977":"Worker (A rating)","68979":"Worker (A rating)","68981":"Worker (A (SME+))","68985":"Worker (A rating)","68989":"Worker (A (SME+))"},"Route":{"68977":"Skilled Worker","68979":"Skilled Worker","68981":"Skilled Worker","68985":"Skilled Worker","68989":"Skilled Worker"}}'

In [9]:
word = "Exis"

{str(i) : search(word).to_dict('records')[i] for i in range(len(search(word)))}

{'0': {'Organisation Name': 'Flexistay Limited',
  'Town/City': 'Croydon',
  'County': 'Surrey',
  'Type & Rating': 'Worker (A rating)',
  'Route': 'Skilled Worker'}}

In [10]:
import requests
import json
import pandas as pd

url = "https://hvpfqi4bob.execute-api.eu-west-2.amazonaws.com/sponsorship-checker"

payload = {
            "word": "Norfolk", 
            "based_on": "org", 
            "route": "Skilled Worker"
          }

payload = json.dumps(payload)
headers = {
  'Content-Type': 'text/plain'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

dd = json.loads(response.text)


{"last_updated": "2023-03-27", "output_table": [{"Organisation Name": "East Norfolk Multi Academy Trust", "Town/City": "Great Yarmouth", "County": "Norfolk", "Type & Rating": "Worker (A rating)", "Route": "Skilled Worker"}, {"Organisation Name": "I Care Service Norfolk Ltd", "Town/City": "Dereham", "County": "Norfolk", "Type & Rating": "Worker (A rating)", "Route": "Skilled Worker"}, {"Organisation Name": "Indian Vegetarian Norfolk Ltd", "Town/City": "Cambridge", "County": "Cambridgeshire", "Type & Rating": "Worker (A rating)", "Route": "Skilled Worker"}, {"Organisation Name": "Norfolk Affinity Care Services Ltd", "Town/City": "Norwich", "County": "Norfolk", "Type & Rating": "Worker (A rating)", "Route": "Skilled Worker"}, {"Organisation Name": "Norfolk and Norwich University Hospitals NHS Foundation Trust", "Town/City": "Norwich", "County": "Norfolk", "Type & Rating": "Worker (A rating)", "Route": "Skilled Worker"}, {"Organisation Name": "Norfolk and Suffolk  NHS Foundation Trust", "T

In [10]:
dd["output_table"]

[{'Organisation Name': 'East Norfolk Multi Academy Trust',
  'Town/City': 'Great Yarmouth',
  'County': 'Norfolk',
  'Type & Rating': 'Worker (A rating)',
  'Route': 'Skilled Worker'},
 {'Organisation Name': 'I Care Service Norfolk Ltd',
  'Town/City': 'Dereham',
  'County': 'Norfolk',
  'Type & Rating': 'Worker (A rating)',
  'Route': 'Skilled Worker'},
 {'Organisation Name': 'Indian Vegetarian Norfolk Ltd',
  'Town/City': 'Cambridge',
  'County': 'Cambridgeshire',
  'Type & Rating': 'Worker (A rating)',
  'Route': 'Skilled Worker'},
 {'Organisation Name': 'Norfolk Affinity Care Services Ltd',
  'Town/City': 'Norwich',
  'County': 'Norfolk',
  'Type & Rating': 'Worker (A rating)',
  'Route': 'Skilled Worker'},
 {'Organisation Name': 'Norfolk and Norwich University Hospitals NHS Foundation Trust',
  'Town/City': 'Norwich',
  'County': 'Norfolk',
  'Type & Rating': 'Worker (A rating)',
  'Route': 'Skilled Worker'},
 {'Organisation Name': 'Norfolk and Suffolk  NHS Foundation Trust',
  'T

In [12]:
data["Route"].unique()

array(['Skilled Worker', 'Creative Worker',
       'Global Business Mobility: Senior or Specialist Worker',
       'Tier 2 Ministers of Religion',
       'Global Business Mobility: UK Expansion Worker',
       'Global Business Mobility: Graduate Trainee',
       'International Sportsperson', 'Charity Worker', 'Religious Worker',
       'Government Authorised Exchange', 'International Agreement',
       'Seasonal Worker', 'Scale-up',
       'Global Business Mobility: Service Supplier',
       'Global Business Mobility: Secondment Worker'], dtype=object)

In [11]:
pd.DataFrame.from_dict(dd["output_table"])

,Organisation Name,Town/City,County,Type & Rating,Route
0,East Norfolk Multi Academy Trust,Great Yarmouth,Norfolk,Worker (A rating),Skilled Worker
1,I Care Service Norfolk Ltd,Dereham,Norfolk,Worker (A rating),Skilled Worker
2,Indian Vegetarian Norfolk Ltd,Cambridge,Cambridgeshire,Worker (A rating),Skilled Worker
3,Norfolk Affinity Care Services Ltd,Norwich,Norfolk,Worker (A rating),Skilled Worker
4,Norfolk and Norwich University Hospitals NHS F...,Norwich,Norfolk,Worker (A rating),Skilled Worker
5,Norfolk and Suffolk NHS Foundation Trust,Norwich,Norfolk,Worker (A rating),Skilled Worker
6,NORFOLK AND SUFFOLK HEALTHCARE LIMITED,Ipswich,Suffolk,Worker (A rating),Skilled Worker
7,Norfolk Community Health & Care NHS Trust,Norwich,Norfolk,Worker (A rating),Skilled Worker
8,Norfolk County Council,Norwich,Norfolk,Worker (A rating),Skilled Worker
9,Norfolk Mead Hotel,Coltishall,Norfolk,Worker (A rating),Skilled Worker


In [12]:
payload

'{\r\n    "word": "Norfolk",\r\n    "based_on": "org",\r\n    "route": "Skilled Worker"\r\n}'